In [ ]:
# !pip install pycaret
# import pycaret
# from pycaret.regression import *
import pandas as pd
import pickle
import numpy as np

In [ ]:
# print(pycaret.__version__)

In [ ]:
# saved_lr = load_model('../input/regression-using-pycaretv2/saved_lr_model')
scaler = pickle.load(open('../input/structured-data-regression-training-pipeline/scaler.pkl','rb'))
pca = pickle.load(open('../input/structured-data-regression-training-pipeline/pca.pkl', 'rb')) #dimensionality reduction

import os, re
regressions = {}

for dirname, _, filenames in os.walk('../input/structured-data-regression-training-pipeline'):
    for filename in filenames:
        m = re.search('[a-z0-9_]+(?=\.pkl)', filename)
        if m:
            if 'lr' in m.group(0):
                n = re.search('[a-z0-9]+(?=\.pkl)', filename)
                regressions[n.group(0)] = pickle.load(open('../input/structured-data-regression-training-pipeline/'+filename, "rb"))

In [ ]:
regressions

In [ ]:
import ubiquant
env = ubiquant.make_env()
iter_test = env.iter_test() 

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    df = pd.DataFrame(columns=np.arange(len(regressions)-1))
    i = 0
    print(df.info())
    test_df.drop(columns=['row_id'], inplace = True)
    test_df = test_df.reset_index(drop = True)
    test_df = scaler.transform(test_df.values)
    test_df = pca.transform(test_df)
    for regression in regressions.keys():
        if regression != 'lr':
            df[i] = regressions[regression].predict(test_df)
            i += 1
            print(df.info())
            
    print(df.info(verbose=True)) 
    print(df.values.shape)
    
    sample_prediction_df['target'] = regressions['lr'].predict(df.values)  # make your predictions here
    env.predict(sample_prediction_df)   # register your predictions